In [1]:
dataset = '22 April to 6 May.xlsx'

# Importing Data And Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

### Reading Our Datasets 

In [3]:
dataset = pd.read_excel(dataset)
df = dataset.copy()

# Cleaning Part

### Extract the year and the month columns


In [4]:
df.rename(columns = {
    'Order Ref/Branch': 'Area'
    , 'Receipt Number':'رقم الفاتوره'
    , 'Order Date':  'Date'
    ,'Order Ref/Sales Team': 'فريق المبيعات' 
    , 'Product/Barcode': 'Barcode'
    ,'Product/Product Category': 'Product Category'
    ,'Total cost': 'Cost'
    , 'Order Ref/Employee/Employee Name': 'Order Ref/Cashier'
    ,'Product/Barcode': 'Barcode'
    ,'Total cost': 'Cost'
    ,'Order Ref/Employee/Employee Name': 'Order Ref/Cashier'
    ,'Product/Sales Price': 'Sales Price'
}, inplace = True)




In [5]:
df['day'] = pd.to_datetime(df['Date']).dt.day
d = list(df['day'].dropna().unique())

df['month'] = pd.to_datetime(df['Date']).dt.month_name()
m = list(df['month'].dropna().unique())

df['year'] = pd.to_datetime(df['Date']).dt.year
y = list(df['year'].dropna().unique())

### Change the directory in order to put all the excel files reports in the day folder

In [6]:
root_path = 'Desktop/'
try:
    os.mkdir(os.path.join(root_path, 'حملات اعلانية'))
    os.chdir(os.path.join(root_path,  'حملات اعلانية'))
except:
    os.chdir(os.path.join(root_path,  'حملات اعلانية'))

### Split product cat Columns

In [7]:
df[['Cat1', 'Product origin', 'cat2', 'cat3', 'cat4']]= df['Product Category'].str.split("/",expand=True)
df.drop(columns= {'Product Category'}, inplace = True)

### Prepare the required Columns

In [8]:
df['discount amount'] = df['Quantity'] * (df['Unit Price'] - df['Subtotal'])
df['discount percentage'] = df['discount amount'] / df['Unit Price']
df['Total discount'] = df['discount amount']  + df['discount percentage']
df['price before discount'] = df['Unit Price'] * df['Quantity']
df['Cost per unit'] = df['Cost'] * df['Quantity']
df['vat'] = df['Subtotal'] / 1.15
df['profit margin'] = df['vat'] - df['Cost'] 
df['Profit margin percentage'] = df['profit margin'] /df['vat']

### Cleaning the values of the branches to be in a better shape

In [9]:
df['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الإحساء' : 'الإحساء',
                             'مبيعات نقاط بيع راحتي الرس' : 'الرس' ,
                             'مبيعات نقاط بيع راحتي بريدة 2' : "بريدة 2",
                             'مبيعات نقاط بيع راحتي الدمام' : "الدمام",
                             'مبيعات نقاط بيع فرع بريدة 1' : 'بريدة 1',
                             'مبيعات نقاط بيع راحتي الطائف 1' : ' الطائف 1',
                              'مبيعات نقاط بيع راحتي البدائع': 'البدائع',
                             'مبيعات نقاط بيع راحتي حائل 1': 'حائل 1',
                             'مبيعات نقاط بيع راحتي حفر الباطن': 'حفر الباطن',
                             'مبيعات نقاط بيع راحتي حائل 2': 'حائل 2',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني',
                             'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل',
                             'مبيعات نقاط بيع فرع تبوك': 'فرع تبوك',
                             'مبيعات نقاط بيع راحتي الرياض': 'الرياض',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الطائف 2': 'راحتي الطائف 2'}, inplace = True)

### Prepare different pages 

#### Final Total

In [10]:
df_final_total = df[df['Product'] == 'الإجمالي النهائي' ]
df.drop(df_final_total.index, inplace = True)


In [11]:
services = df[df['Cat1'] == 'خدمات ']
df_s = df.copy()
df.drop(services.index, inplace = True)

In [12]:
branch = df[df['فريق المبيعات'] == 'مبيعات فرع الجملة' ]
df.drop(branch.index, inplace = True)

# Analytical Part

## Your Choices (Branch and date)

In [13]:
compaign_name= input("Enter the compaign name")


Enter the compaign nameالماركتنج - قصر البيان حائل - عروض حائل


In [14]:
print(df['فريق المبيعات'].unique())
branch = input('Enter the branch name')

['الدمام' 'حائل 2' 'الرس' 'فرع تبوك' ' الطائف 1' 'بريدة 1' 'حفر الباطن'
 'حائل 1' 'قصر البيان حائل' 'بريدة 2' 'الإحساء' 'البدائع' 'راحتي الطائف 2'
 'الرياض' 'قصر البيان الإلكتروني' 'متجر راحتي الالكتروني']
Enter the branch nameقصر البيان حائل


In [15]:
import datetime
begin_day_before = int(input('Enter the beginning of 3 days before'))
begin_month_before = int(input('Enter the month'))

sdate_before = datetime.date(2024, begin_month_before, begin_day_before)

end_day_before = int(input('Enter the end of 3 days before'))
end_month_before = int(input('Enter the month'))
edate_before = datetime.date(2024,end_month_before,end_day_before) 

Enter the beginning of 3 days before24
Enter the month4
Enter the end of 3 days before27
Enter the month4


In [16]:
begin_day_after = int(input('Enter the beginning of 3 days after'))
begin_month_after = int(input('Enter the month'))

sdate_after = datetime.date(2024, begin_month_after ,begin_day_after)

end_day_after = int(input('Enter the end of 3 days after'))
end_month_after = int(input('Enter the month'))
edate_after = datetime.date(2024, end_month_after, end_day_after )

Enter the beginning of 3 days after28
Enter the month4
Enter the end of 3 days after1
Enter the month5


## Before

### Product Analysis

In [29]:
df2 = df.copy()
df2['Date'] = pd.to_datetime(df['Date']).dt.date
df2 = df2[df2.Date.between(sdate_before, edate_before)]
df2 = df2[df2['فريق المبيعات'] == branch]
df_s = df_s[df_s['فريق المبيعات'] == branch]

In [31]:
try:
    branch_team = df2[df2['فريق المبيعات'] == branch]
    Total_sold = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['Quantity']), aggfunc = 'sum')
    Total_sold.rename(columns ={'Quantity':'sold units'}, inplace = True)
    Total_sold = Total_sold.sort_values(by= 'sold units', ascending = False).head(10)
except:
    pass

mean_discounts = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['discount amount']), aggfunc = 'mean')
mean_discounts.rename(columns ={'discount amount':'Total discounts'}, inplace = True)        
    

total_sales = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['Subtotal']), aggfunc = 'sum')
total_sales.rename(columns ={'Subtotal':'Total sales'}, inplace = True)



total_profit = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['profit margin']), aggfunc = 'sum')
total_profit.rename(columns ={'profit margin':'Total Profit'}, inplace = True)

total_vat = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['vat']), aggfunc = 'sum')
total_vat.rename(columns ={'vat':'Total Vat'}, inplace = True)



margin = total_profit.merge(total_vat, on= ['فريق المبيعات', 'Product'], how= 'inner')
margin['هامش الربح'] = margin['Total Profit'] / margin['Total Vat']




sales_profit = Total_sold.merge(mean_discounts, on= ['فريق المبيعات', 'Product'], how= 'inner')
sales_profit = sales_profit.merge(total_sales, on= ['فريق المبيعات', 'Product'], how= 'inner')
sales_profit = sales_profit.merge(margin['هامش الربح'], on= ['فريق المبيعات', 'Product'], how= 'inner')


sales_profit.reset_index(inplace = True)
sales_profit.reset_index(inplace = True)

sales_profit.rename(columns = {
    'index': 'الترتيب',
    'Product': 'المنتج',
    'sold units': "عدد الوحدات",
    'Total discounts': "إجمالي الخصومات",
    'Total sales': "اجمالي قيمة المبيعات",    
}, inplace = True)

sales_profit.sort_values(by= ['عدد الوحدات', 'اجمالي قيمة المبيعات'], ascending = [False, False], inplace = True)

sales_profit['الترتيب'] = sales_profit['الترتيب'] + 1 
sales_profit.drop(columns =  {'فريق المبيعات'}, inplace = True)
sales_profit.set_index("الترتيب", inplace = True)

sales_profit.loc['الإجمالي', 'المنتج'] =  sales_profit['المنتج'].count()
sales_profit.loc['الإجمالي', 'عدد الوحدات'] =  sales_profit['عدد الوحدات'].sum()
sales_profit.loc['الإجمالي', 'إجمالي الخصومات'] =  sales_profit['إجمالي الخصومات'].sum()
sales_profit.loc['الإجمالي', 'اجمالي قيمة المبيعات'] =  sales_profit['اجمالي قيمة المبيعات'].sum()



sales_profit.to_excel(compaign_name + '.xlsx' , sheet_name= 'المنتجات قبل' )

### Days Analysis

In [32]:
data_sales = pd.DataFrame()
margin_p = 0 
for day in list(df2['day'].unique()):
    try:
        branch_team = df2[df2['day'] == day]
        invoices_num = pd.pivot_table(branch_team, index=['فريق المبيعات'], values=(['رقم الفاتوره']), aggfunc = 'nunique')
        invoices_num.rename(columns ={'رقم الفاتوره':'Number of invoices'}, inplace = True)
    except:
        pass
    
    total_sales = pd.pivot_table(df_s[df_s['day'] == day], index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum')
    total_sales.rename(columns ={'Subtotal':'Total sales'}, inplace = True)

    Margin_p = (total_sales['Total sales'][0] - margin_p) /margin_p    
    margin_p = total_sales['Total sales']
    
    total_sold = pd.pivot_table(branch_team, index=['فريق المبيعات'], values=(['Quantity']), aggfunc = 'sum')
    total_sold.rename(columns ={'Quantity':'sold units'}, inplace = True)
    
    total_profit = pd.pivot_table(df_s[df_s['day'] == day], index=['فريق المبيعات'], values=(['profit margin']), aggfunc = 'sum')
    total_profit.rename(columns ={'profit margin':'Total Profit'}, inplace = True)
    total_vat = pd.pivot_table(df_s[df_s['day'] == day], index=['فريق المبيعات'], values=(['vat']), aggfunc = 'sum')
    
    total_vat.rename(columns ={'vat':'Total Vat'}, inplace = True)
    margin = total_profit.merge(total_vat, on= ['فريق المبيعات'], how= 'inner')
    margin['هامش الربح'] = margin['Total Profit'] / margin['Total Vat']
    
    sales_profit = total_sold.merge(invoices_num, on= ['فريق المبيعات'], how= 'inner')
    sales_profit = sales_profit.merge(total_sales, on= ['فريق المبيعات'], how= 'inner')
    sales_profit = sales_profit.merge(margin, on= ['فريق المبيعات'], how= 'inner')
    sales_profit['التاريخ'] = branch_team['Date'].unique()[0]
    sales_profit['النغير في هامش الربح']= Margin_p
    data_sales =pd.concat([data_sales, sales_profit.loc[: ]])
    
data_sales.reset_index(inplace = True)
data_sales.reset_index(inplace = True)
data_sales.rename(columns = {'index': "الترتيب",
                            'sold units': 'عدد القطع المباعة',
                            'Total sales': 'اجمالي المبيعات (شامل الخدمات)',
                            'Number of invoices': 'عدد الفواتير'}, inplace = True)
data_sales.drop(columns = {'فريق المبيعات'}, inplace = True)
data_sales['الترتيب'] = data_sales['الترتيب'] + 1
data_sales.set_index('الترتيب', inplace = True)

data_sales.loc['الإجمالي', 'التاريخ'] =  data_sales['هامش الربح'].count()
data_sales.loc['الإجمالي', 'عدد القطع المباعة'] =  data_sales['عدد القطع المباعة'].sum()
data_sales.loc['الإجمالي', 'عدد الفواتير'] = data_sales['عدد الفواتير'].sum()
data_sales.loc['الإجمالي', 'اجمالي المبيعات (شامل الخدمات)'] =  data_sales['اجمالي المبيعات (شامل الخدمات)'].sum()
data_sales.loc['الإجمالي','هامش الربح'] =data_sales['Total Profit'].sum() / data_sales['Total Vat'].sum()
data_sales.drop(columns = {'Total Profit', 'Total Vat'}, inplace = True)
data_sales = data_sales[['التاريخ', 'عدد الفواتير', 'اجمالي المبيعات (شامل الخدمات)', 'هامش الربح', 'عدد القطع المباعة', 'النغير في هامش الربح']]
days_before = data_sales['اجمالي المبيعات (شامل الخدمات)'].sum()


with pd.ExcelWriter(compaign_name + '.xlsx', engine='openpyxl', mode='a') as writer:
    data_sales.to_excel(writer, sheet_name= 'الايام قبل')

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_11888\1342146987.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  Margin_p = (total_sales['Total sales'][0] - margin_p) /margin_p


## After

### Product Analysis


In [33]:
df3 = df.copy()
df3['Date'] = pd.to_datetime(df['Date']).dt.date
df3 = df3[df3['فريق المبيعات'] == branch]
df3 = df3[df3.Date.between(sdate_after, edate_after)]
try:
    branch_team = df3.copy()
    Total_sold = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['Quantity']), aggfunc = 'sum')
    Total_sold.rename(columns ={'Quantity':'sold units'}, inplace = True)
    Total_sold = Total_sold.sort_values(by= 'sold units', ascending = False).head(10)
except:
    pass

mean_discounts = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['discount amount']), aggfunc = 'mean')
mean_discounts.rename(columns ={'discount amount':'Total discounts'}, inplace = True)        
    

total_sales = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['Subtotal']), aggfunc = 'sum')
total_sales.rename(columns ={'Subtotal':'Total sales'}, inplace = True)

total_profit = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['profit margin']), aggfunc = 'sum')
total_profit.rename(columns ={'profit margin':'Total Profit'}, inplace = True)

total_vat = pd.pivot_table(branch_team, index=['فريق المبيعات', 'Product'], values=(['vat']), aggfunc = 'sum')
total_vat.rename(columns ={'vat':'Total Vat'}, inplace = True)


margin = total_profit.merge(total_vat, on= ['فريق المبيعات', 'Product'], how= 'inner')
margin['هامش الربح'] = margin['Total Profit'] / margin['Total Vat']

sales_profit = Total_sold.merge(mean_discounts, on= ['فريق المبيعات', 'Product'], how= 'inner')
sales_profit = sales_profit.merge(total_sales, on= ['فريق المبيعات', 'Product'], how= 'inner')
sales_profit = sales_profit.merge(margin['هامش الربح'], on= ['فريق المبيعات', 'Product'], how= 'inner')


sales_profit.reset_index(inplace = True)
sales_profit.reset_index(inplace = True)

sales_profit.rename(columns = {
    'index': 'الترتيب',
    'Product': 'المنتج',
    'sold units': "عدد الوحدات",
    'Total discounts': "إجمالي الخصومات",
    'Total sales': "اجمالي قيمة المبيعات",    
}, inplace = True)

sales_profit.sort_values(by= ['عدد الوحدات', 'اجمالي قيمة المبيعات'], ascending = [False, False], inplace = True)


sales_profit['الترتيب'] = sales_profit['الترتيب'] + 1 
sales_profit.drop(columns =  {'فريق المبيعات'}, inplace = True)
sales_profit.set_index("الترتيب", inplace = True)

sales_profit.loc['الإجمالي', 'المنتج'] =  sales_profit['المنتج'].count()
sales_profit.loc['الإجمالي', 'عدد الوحدات'] =  sales_profit['عدد الوحدات'].sum()
sales_profit.loc['الإجمالي', 'إجمالي الخصومات'] =  sales_profit['إجمالي الخصومات'].sum()
sales_profit.loc['الإجمالي', 'اجمالي قيمة المبيعات'] =  sales_profit['اجمالي قيمة المبيعات'].sum()

with pd.ExcelWriter(compaign_name + '.xlsx', engine='openpyxl', mode='a') as writer:
    sales_profit.to_excel(writer, sheet_name= 'المنتجات بعد')


### Days Analysis

In [34]:
data_sales = pd.DataFrame()
for day in list(df3['day'].unique()):
    try:
        branch_team = df3[df3['day'] == day]
        invoices_num = pd.pivot_table(branch_team, index=['فريق المبيعات'], values=(['رقم الفاتوره']), aggfunc = 'nunique')
        invoices_num.rename(columns ={'رقم الفاتوره':'Number of invoices'}, inplace = True)
    except:
        pass
            
    total_sales = pd.pivot_table(df_s[df_s['day'] == day], index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum')
    total_sales.rename(columns ={'Subtotal':'Total sales'}, inplace = True)
   
    Margin_p = (total_sales['Total sales'][0] - margin_p) /margin_p    
    margin_p = total_sales['Total sales']
        
    total_sold = pd.pivot_table(branch_team, index=['فريق المبيعات'], values=(['Quantity']), aggfunc = 'sum')
    total_sold.rename(columns ={'Quantity':'sold units'}, inplace = True)
    
    total_profit = pd.pivot_table(df_s[df_s['day'] == day], index=['فريق المبيعات'], values=(['profit margin']), aggfunc = 'sum')
    total_profit.rename(columns ={'profit margin':'Total Profit'}, inplace = True)
    total_vat = pd.pivot_table(df_s[df_s['day'] == day], index=['فريق المبيعات'], values=(['vat']), aggfunc = 'sum')
    total_vat.rename(columns ={'vat':'Total Vat'}, inplace = True)
    margin = total_profit.merge(total_vat, on= ['فريق المبيعات'], how= 'inner')
    margin['هامش الربح'] = margin['Total Profit'] / margin['Total Vat']
    
    sales_profit = total_sold.merge(invoices_num, on= ['فريق المبيعات'], how= 'inner')
    sales_profit = sales_profit.merge(total_sales, on= ['فريق المبيعات'], how= 'inner')
    sales_profit = sales_profit.merge(margin, on= ['فريق المبيعات'], how= 'inner')
    sales_profit['التاريخ'] = branch_team['Date'].unique()[0]
    sales_profit['النغير في هامش الربح']= Margin_p
    data_sales =pd.concat([data_sales, sales_profit.loc[: ]])
    
data_sales.reset_index(inplace = True)
data_sales.reset_index(inplace = True)
data_sales.rename(columns = {'index': "الترتيب",
                            'sold units': 'عدد القطع المباعة',
                            'Total sales': 'اجمالي المبيعات (شامل الخدمات)',
                            'Number of invoices': 'عدد الفواتير'}, inplace = True)
data_sales.drop(columns = {'فريق المبيعات'}, inplace = True)
data_sales['الترتيب'] = data_sales['الترتيب'] + 1
data_sales.set_index('الترتيب', inplace = True)

data_sales.loc['الإجمالي', 'التاريخ'] =  data_sales['هامش الربح'].count()
data_sales.loc['الإجمالي', 'عدد القطع المباعة'] =  data_sales['عدد القطع المباعة'].sum()
data_sales.loc['الإجمالي', 'عدد الفواتير'] = data_sales['عدد الفواتير'].sum()
data_sales.loc['الإجمالي', 'اجمالي المبيعات (شامل الخدمات)'] =  data_sales['اجمالي المبيعات (شامل الخدمات)'].sum()
data_sales.loc['الإجمالي','هامش الربح'] =data_sales['Total Profit'].sum() / data_sales['Total Vat'].sum()
data_sales.loc['الإجمالي','النغير في هامش الربح'] =(data_sales['اجمالي المبيعات (شامل الخدمات)'].sum() -  days_before)/ days_before

data_sales.drop(columns = {'Total Profit', 'Total Vat'}, inplace = True)
data_sales = data_sales[['التاريخ', 'عدد الفواتير', 'اجمالي المبيعات (شامل الخدمات)', 'هامش الربح', 'عدد القطع المباعة', 'النغير في هامش الربح']]

with pd.ExcelWriter(compaign_name + '.xlsx', engine='openpyxl', mode='a') as writer:
    data_sales.to_excel(writer, sheet_name= 'الايام بعد')